# Plotting: 1D and 2D data

This notebook show how to visualize 1D and 2d simulation data.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

## Plotting 1D data

1D and 2D data are loaded in exactly the same way as 3D simulations,
by creating a `Dataset` and calling the `load()` method.

In [ ]:
path = "osyrisdata/sod"
data1d = osyris.RamsesDataset(2, path=path).load()
data1d

### Line profile

To plot a simple line profile of density as a function of position,
Osyris provides the `plot` function which is analogous to Matplotlib's
[plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) function.

In [ ]:
data1d = data1d["mesh"]

osyris.plot(data1d["position_x"], data1d["density"])

It is also possible to send just a single array to the `plot` function,
in which case it will use it as `y` values and replace the `x` axis with integer numbers spanning the length of the array:

In [ ]:
osyris.plot(data1d["density"])

Note that in this case, the order in which the data points are plotted is just the order in which they appear in the output files,
i.e. it will most probably not make physical sense, as it depends (among other things) on load-balancing across cpus.

However, these plots can be useful for debugging purposes,
when one wishes to visually inspect the range of values spanned by a variable.

### Styling lines

Because the function is based on Matplotlib's implementation,
it also supports Matplotlib's styling arguments.
For instance, to plot using red marker instead of a solid line,
you can do

In [ ]:
osyris.plot(
    data1d["position_x"],
    data1d["density"],
    marker="o",
    ls="None",
    color="red",
)

### Logarithmic scales

Osyris does provide some convenience arguments in addition to Matplotlib's functionalty.
For example, to set a logarithmic scale on the x and y axes, use the `logx` and `logy` arguments

In [ ]:
osyris.plot(
    data1d["position_x"],
    data1d["density"],
    marker="o",
    ls="None",
    logx=True,
    logy=True,
)

### Multiple lines

It is also possible to over-plot multiple lines in one go,
by supplying more than one array for the `y` values.

However, note that to be able to plot the variables on a single axis,
they must all have the same unit.

In [ ]:
try:
    osyris.plot(
        data1d["position_x"],
        data1d["density"],
        data1d["pressure"],
    )
except Exception as e:
    print(e)

To be able to show an example, we fake a new density variable.

In [ ]:
data1d["new_density"] = data1d["density"] * 2
osyris.plot(
    data1d["position_x"],
    data1d["density"],
    data1d["new_density"],
)

By default, the same x coordinate will be used for both sets of y values.
It is however possible to plot on the same axes two variables with different x coordinates.

In this case, the two entries must be dicts, and they must contain at least the entries `'x'` and `'y'`.
This is useful in the case where one wishes to compare two outputs from different times,
which do not contain the same number of mesh cells (and hence cannot have the same x coordinate).

In [ ]:
old1d = osyris.RamsesDataset(1, path=path).load()["mesh"]

osyris.plot(
    {"x": data1d["position_x"], "y": data1d["density"]},
    {"x": old1d["position_x"], "y": old1d["density"]},
    marker="o",
    ls="None",
)

## Subplots / tiled plots

Osyris has no built-in support for subplots (also known as tiled plots).
Instead, we leverage Matplotlib's ability to create such layouts.
Osyris plots are then inserted into the Matplotlib axes, using the `ax` argument.

In the example below, we create four panels and insert various plots.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 9))

osyris.plot(
    data1d["position_x"],
    data1d["density"],
    marker="o",
    ls="None",
    ax=ax[0, 0],
)
osyris.plot(
    data1d["position_x"],
    data1d["velocity_x"],
    color="green",
    ax=ax[0, 1],
)
osyris.plot(data1d["position_x"], data1d["pressure"], color="red", ax=ax[1, 0])
osyris.plot(
    data1d["position_x"],
    data1d["level"],
    color="black",
    ls="dotted",
    ax=ax[1, 1],
)

## Plotting 2D data

This section briefly shows how to make images of 2D data with the `map` function.
For a full description of all the options available, see [Plotting: spatial maps](plotting_maps.ipynb).

In [ ]:
data2d = osyris.RamsesDataset(2, path="osyrisdata/sedov").load()

In [ ]:
data2d["mesh"]

To make an image of the gas density, simply use

In [ ]:
osyris.map(data2d["mesh"].layer("density"), norm="log", cmap="jet")

A map of the AMR level is obtained with

In [ ]:
osyris.map(data2d["mesh"].layer("level"))

The size of the viewport can be adjusted with the `dx` argument:

In [ ]:
osyris.map(
    data2d["mesh"].layer("density"),
    dx=2 * osyris.units("cm"),
    cmap="jet",
    norm="log",
)

Finally, you can also use an additional layer to overlay velocity vectors

In [ ]:
osyris.map(
    data2d["mesh"].layer("density", norm="log"),
    data2d["mesh"].layer("velocity", mode="vec", color="black"),
    cmap="jet",
)